In [1]:
import csv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sn
from sklearn.linear_model import Perceptron
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
import sys
import time
import progressbar



In [2]:
X = []
y = []
#Import Pulsar dataset from csv
with open("pulsar_stars.csv") as csv_file:
    csv_read = csv.reader(csv_file, delimiter = ',')
    for line in csv_read:
        #print(line)
        feat = []
        binClass = []
        i = 0
        while i<len(line):
            #print(line[i],i)
            if i == 8:
                num = float(line[i])
                #print(num)
                binClass.append(num)
            else:
                num = float(line[i])
                feat.append(num)
            i+=1

        X.append(feat)
        y.append(binClass)

        

#print(X[1])
#print(y[1])
#print(y)

y = np.reshape(y,len(y))
scaler = StandardScaler()
scaler.fit(X)
scaler.transform(X)

array([[ 1.14931702,  1.33483164, -0.66956953, ..., -0.37062547,
        -0.07279778, -0.28743812],
       [-0.3341682 ,  1.80226533, -0.01178476, ..., -0.5889241 ,
         0.50442694,  0.21158145],
       [-0.31437216, -1.05332222, -0.14523256, ..., -0.23532816,
        -0.12599609, -0.39137346],
       ...,
       [ 0.3218423 ,  1.95621968, -0.2993338 , ...,  1.67156847,
        -1.28807874, -0.94133005],
       [ 0.13362759,  1.07450972, -0.26005007, ..., -0.66485697,
         0.37825656,  0.27584987],
       [-2.10576204,  5.73546965,  0.87267394, ...,  1.97154554,
        -2.19732744, -0.97105168]])

In [3]:
scores = []
#Define Multilayer Perceptron Model
hyperparam_val = []
hidden_lays = [(4,4,3,2), (7, 4), (8, 3, 2)]
for epoch in range(9):
    if not (epoch == 0):
        for learn_rate in range(8):
            #(hidden units, learning rate, c)
            for num_hid in range(len(hidden_lays)):
                hyperparam_val.append((0.0001*10**learn_rate,100*epoch,num_hid))

bar = progressbar.ProgressBar(maxval=len(hyperparam_val))

count = 0
for param in hyperparam_val:
    #print(param)
    mlp = MLPClassifier(activation='tanh', solver = 'adam', alpha = param[0], hidden_layer_sizes=(8,4,2), max_iter=param[1])
    mlp.fit(X,y)
    #print(sp.get_params())
    yP = mlp.predict(X)
    #print(sp.score(X,y))
    #scores.append((param,mlp.score(X,y)))
    score = 0
    for i in range(len(yP)):
        if y[i] == yP[i]:
            score += 1
        
    scores.append((param[0], param[1], param[2], (score/len(yP) * 100)))
    
    count += 1
    sys.stdout.flush()
    bar.update(count)
    
bar.finish()



/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
N/A% (0 of 192) |                        | Elapsed Time: 0:00:00 ETA:  --:--:--/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
  1% (2 of 192) |                        | Elapsed Time: 0:00:03 ETA:   0:05:18/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
  2% (4 of 192) |                        | Elapsed Time: 0:00:10 ETA:   0:10:10/anaconda3/lib/p

In [4]:
scores2 = []
bar = progressbar.ProgressBar(maxval=len(hyperparam_val))

count = 0
for param in hyperparam_val:
    #print(param)
    mlp2 = MLPClassifier(activation='tanh', solver = 'lbfgs', alpha = param[0], hidden_layer_sizes=(8,4,2), max_iter=param[1])
    mlp2.fit(X,y)
    #print(sp.get_params())
    yP = mlp2.predict(X)
    #print(sp.score(X,y))
    #scores.append((param,mlp.score(X,y)))
    score = 0
    for i in range(len(yP)):
        if y[i] == yP[i]:
            score += 1
        
    scores2.append((param[0], param[1], param[2], (score/len(yP) * 100)))
    
    count += 1
    sys.stdout.flush()
    bar.update(count)
    
bar.finish()




100% (192 of 192) |######################| Elapsed Time: 0:05:09 Time:  0:05:09


In [5]:
max = 0
maxInfo = ()
for score in scores:
    if score[3] > max:
        max = score[3]
        maxInfo = score

print(maxInfo)


max = 0
maxInfo = ()
for score in scores:
    if score[3] > max:
        max = score[3]
        maxInfo = score

print(maxInfo)



(0.1, 200, 1, 98.05006145938093)
(0.1, 200, 1, 98.05006145938093)


In [12]:
bestMLP = MLPClassifier(alpha = maxInfo[0],hidden_layer_sizes=hidden_lays[maxInfo[2]],max_iter= maxInfo[1])
bestMLP.fit(X, y)
yP = bestMLP.predict(X)
print(confusion_matrix(y, yP))



[[16169    90]
 [  320  1319]]


In [11]:
baggedSP = BaggingClassifier(base_estimator= bestMLP, n_estimators= 15, max_samples= 500, max_features= 8, bootstrap= True)
baggedSP.fit(X,y)
print('Bagged Accuracy:',baggedSP.score(X,y))
score = 0
for i in range(len(yP)):
    if y[i] == yP[i]:
        score += 1
print(score/len(yP))
#sequencedSp = BaggingClassifier(base_estimator= adaSP, n_estimators= 15, max_samples= 500, max_features= 8, bootstrap= True)
#sequencedSp.fit(X,y)
#print('Sequenced Accuracy:', sequencedSp.score(X,y))




/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/ana

Bagged Accuracy: 0.9719521734271985
0.9767571795731367


/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [14]:
print("hi")

hi
